In [1]:
features = [
    'lead_speed_diff',
    'hp_advantage_seen','mons_revealed_diff','team_status_diff','end_boost_diff',
    'total_damage_dealt','total_healing_done','status_turns',
    'first_faint_turn','total_stats_diff','damage_diff_turn10',
    'damage_diff_turn20','damage_diff_turn25','damage_diff_turn30',
    'hp_trend_diff','feat_switch_diff','feat_aggression_diff','hp_diff_std',
    'hp_diff_range','momentum_shift_turn','comeback_score','early_sustain',
    'status_balance','boost_volatility','boost_trend','move_power_diff',
    'move_diversity_diff','stall_ratio','aggression_index',
    'stats_speed_interaction',
    'hp_vs_stats_ratio','damage_ratio_turn25_30','damage_ratio_turn20_25',
    'damage_ratio_turn10_20','damage_ratio_turn10_30',
    'atk_def_ratio_p1','atk_def_ratio_p2','hp_speed_interaction_lead','hp_def_ratio_p1',
    'hp_def_ratio_p2','p1_hp_mean','p2_hp_mean','hp_diff_mean','hp_diff_last',
    'p1_boost_mean','p2_boost_mean','boost_diff_mean','p1_status_total',
    'p2_status_total','momentum_flips','p1_aggression','p2_aggression',
    'aggression_diff','feat_team_emb_sim',
    'lead_type_adv','meta_diff','feat_status_diff_inflicted','status_setup_diff',
    
]


In [2]:
from main import load_data
from Features.features_olya import create_advanced_features_gen2
import os
import pandas as pd
from utils.load_json import load_jsonl
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

test_size=0.2
random_state=42
train_df, test_df = load_data()

# Feature engineering
X_train_features = create_advanced_features_gen2(train_df)
X_test_features = create_advanced_features_gen2(test_df)

# Target
y_train = train_df.set_index('battle_id')['player_won']

# Train/val split
X_train_split, X_val_split, y_train_split, y_val_split = train_test_split(
    X_train_features,
    y_train,
    test_size=test_size,
    random_state=random_state,
    stratify=y_train
)

print("Shapes:")
print(X_train_split.shape, X_val_split.shape, y_train_split.shape, y_val_split.shape)

/home/esquilotorto/.pyenv/versions/3.12.0/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✓ Local environment detected. Loading data from: Data
Riga 4877 rimossa con successo.
✓ train.jsonl loaded successfully. Shape: (9996, 5)
✓ test.jsonl loaded successfully. Shape: (5000, 4)


Generating advanced features: 100%|██████████| 5000/5000 [00:05<00:00, 986.99it/s] 


Shapes:
(7996, 84) (2000, 84) (7996,) (2000,)


In [3]:
from Models.pipeline import get_pipeline

"""
Available models and recommended scaler usage:

1. Logistic Regression ('logistic')
   - Recommended scaler: RobustScaler (default 'auto')
   - Key parameters: C, penalty ('l1', 'l2'), class_weight

2. Random Forest ('random_forest')
   - Recommended scaler: RobustScaler (default 'auto')
   - Key parameters: n_estimators, max_depth, min_samples_split, min_samples_leaf, max_features

3. XGBoost ('xgboost')
   - Recommended scaler: RobustScaler (default 'auto')
   - Key parameters: n_estimators, max_depth, learning_rate, subsample, colsample_bytree, gamma

4. LightGBM ('lightgbm')
   - Recommended scaler: StandardScaler (default 'auto')
   - Key parameters: n_estimators, num_leaves, learning_rate, max_depth, feature_fraction, bagging_fraction, min_child_samples, lambda_l1, lambda_l2

5. CatBoost ('catboost')
   - Recommended scaler: StandardScaler (default 'auto')
   - Key parameters: depth, learning_rate, iterations, l2_leaf_reg, random_seed, task_type

6. Gradient Boosting ('gradient_boost')
   - Recommended scaler: RobustScaler (default 'auto')
   - Key parameters: n_estimators, max_depth, learning_rate, min_samples_split, min_samples_leaf, subsample
"""

from Models.pipeline import get_pipeline

"""
Pipelines for all supported models.
Each pipeline is created with the recommended scaler unless you override it.
"""

# 1. Logistic Regression
pipeline_logistic = get_pipeline(
    model_name='logistic',
    numerical_features=features,
    scaler='robust' 
)

# 2. Random Forest
pipeline_random_forest = get_pipeline(
    model_name='random_forest',
    numerical_features=features,
    scaler='robust' 
)

# 3. XGBoost
pipeline_xgb = get_pipeline(
    model_name='xgboost',
    numerical_features=features,
    scaler='robust' 
)

# 4. LightGBM
pipeline_lgbm = get_pipeline(
    model_name='lightgbm',
    numerical_features=features,
    scaler='standard'
)

# 5. CatBoost
pipeline_catboost = get_pipeline(
    model_name='catboost',
    numerical_features=features,
    scaler='standard'
)

# 6. Gradient Boosting
pipeline_gradient_boost = get_pipeline(
    model_name='gradient_boost',
    numerical_features=features,
    scaler='robust' 
)

# If you want all of them in one dict:
all_pipelines = {
    "logistic": pipeline_logistic,
    "random_forest": pipeline_random_forest,
    "xgboost": pipeline_xgb,
    "lightgbm": pipeline_lgbm,
    "catboost": pipeline_catboost,
    "gradient_boost": pipeline_gradient_boost
}

In [4]:
from sklearn.metrics import accuracy_score

# Prepare data
X_train_clean = X_train_split
X_val_clean = X_val_split

results = {}

for name, pipeline in all_pipelines.items():
    print(f"\nTraining {name}...")

    # Fit
    pipeline.fit(X_train_clean, y_train_split)

    # Predict
    preds = pipeline.predict(X_val_clean)

    # Evaluate
    acc = accuracy_score(y_val_split, preds)
    results[name] = acc

    print(f"{name} validation accuracy: {acc:.4f}")

print("\nSummary of results:")
for name, acc in results.items():
    print(f"{name}: {acc:.4f}")



Training logistic...
logistic validation accuracy: 0.7705

Training random_forest...
random_forest validation accuracy: 0.8120

Training xgboost...
xgboost validation accuracy: 0.8080

Training lightgbm...
[LightGBM] [Info] Number of positive: 3998, number of negative: 3998
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001293 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 5808
[LightGBM] [Info] Number of data points in the train set: 7996, number of used features: 44
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
lightgbm validation accuracy: 0.8065

Training catboost...
catboost validation accuracy: 0.8175

Training gradient_boost...
gradient_boost validation accuracy: 0.8120

Summary of results:
logistic: 0.7705
random_forest: 0.8120
xgboost: 0.8080
lightgbm: 0.8065
catboost: 0.8175
gradient_boost: 

In [5]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

X_train_clean = X_train_split
X_val_clean   = X_val_split

results = {}

for name, pipeline in all_pipelines.items():
    print(f"\n=== {name.upper()} ===")

    # Fit
    pipeline.fit(X_train_clean, y_train_split)

    # Predict labels
    preds = pipeline.predict(X_val_clean)

    # Predict probabilities (if available)
    try:
        probs = pipeline.predict_proba(X_val_clean)[:, 1]
    except:
        probs = None

    # Accuracy
    acc = accuracy_score(y_val_split, preds)
    results[name] = acc
    print(f"Validation Accuracy: {acc:.4f}")

    # Classification report
    print("\nClassification Report:")
    print(classification_report(y_val_split, preds))

    # Confusion matrix
    print("Confusion Matrix:")
    print(confusion_matrix(y_val_split, preds))

    # Store predictions & probs if you want to use later
    results[name] = {
        "accuracy": acc,
        "preds": preds,
        "probs": probs
    }

print("\n=== SUMMARY ===")
for name, info in results.items():
    print(f"{name}: {info['accuracy']:.4f}")



=== LOGISTIC ===
Validation Accuracy: 0.7705

Classification Report:
              precision    recall  f1-score   support

       False       0.77      0.76      0.77      1000
        True       0.77      0.78      0.77      1000

    accuracy                           0.77      2000
   macro avg       0.77      0.77      0.77      2000
weighted avg       0.77      0.77      0.77      2000

Confusion Matrix:
[[763 237]
 [222 778]]

=== RANDOM_FOREST ===
Validation Accuracy: 0.8120

Classification Report:
              precision    recall  f1-score   support

       False       0.80      0.83      0.82      1000
        True       0.82      0.79      0.81      1000

    accuracy                           0.81      2000
   macro avg       0.81      0.81      0.81      2000
weighted avg       0.81      0.81      0.81      2000

Confusion Matrix:
[[832 168]
 [208 792]]

=== XGBOOST ===
Validation Accuracy: 0.8080

Classification Report:
              precision    recall  f1-score   supp

In [6]:
from paramethers.cat_grid import param_grid as catboost_param_grid
from paramethers.gb_grid import param_grid as gradientboost_param_grid
from paramethers.lgb_grid import param_grid as lightgbm_param_grid
from paramethers.log_grid import param_grid as logistic_param_grid
from paramethers.rf_grid import param_grid as randomforest_param_grid
from paramethers.xgb_grid import param_grid as xgboost_param_grid


from optimisers.gridsearch_optimizer import run_grid_search
from optimisers.optuna_optimizer import optimize_optuna
from optimisers.randomsearch_optimizer import run_random_search


In [7]:
xgboost_param_grid
catboost_param_grid
gradientboost_param_grid
lightgbm_param_grid

{'classifier__boosting_type': ['dart'],
 'classifier__n_estimators': [400, 500, 600, 700, 800],
 'classifier__learning_rate': [0.01, 0.03, 0.05, 0.08, 0.1, 0.15, 0.2],
 'classifier__num_leaves': [31, 63, 127, 255],
 'classifier__max_depth': [-1, 3, 5, 8, 10, 12],
 'classifier__feature_fraction': [0.7, 0.8, 0.9, 1.0],
 'classifier__bagging_fraction': [0.7, 0.8, 0.9, 1.0],
 'classifier__min_child_samples': [5, 10, 20, 30, 50],
 'classifier__lambda_l1': [0.001, 0.01, 0.1, 1.0, 5.0, 10.0],
 'classifier__lambda_l2': [0.001, 0.01, 0.1, 1.0, 5.0, 10.0]}

In [ ]:
# --- XGBOOST ---
best_model, best_params_xgb, best_score_xgb = optimize_optuna(
    lambda: pipeline_xgb,
    X_train_split,
    y_train_split,
    X_val_split,
    y_val_split,
    xgboost_param_grid,
    n_trials=50
)
print("\nBest XGBoost Accuracy:", best_score_xgb)
print("Best XGBoost Params:", best_params_xgb)


# --- LIGHTGBM ---
best_lgbm_model, best_params_lgb, best_score_lgb = optimize_optuna(
    lambda: pipeline_lgbm,
    X_train_split,
    y_train_split,
    X_val_split,
    y_val_split,
    lightgbm_param_grid,
    n_trials=50
)
print("\nBest LightGBM Accuracy:", best_score_lgb)
print("Best LightGBM Params:", best_params_lgb)

'''
# --- CATBOOST ---
best_cat_model, best_params_cat, best_score_cat = optimize_optuna(
    lambda: pipeline_catboost,
    X_train_split,
    y_train_split,
    X_val_split,
    y_val_split,
    catboost_param_grid,
    n_trials=1
)
print("\nBest CatBoost Accuracy:", best_score_cat)
print("Best CatBoost Params:", best_params_cat)
'''
'''
# --- GRADIENT BOOSTING ---
best_gb_model, best_params_gb, best_score_gb = optimize_optuna(
    lambda: pipeline_gradient_boost,
    X_train_split,
    y_train_split,
    X_val_split,
    y_val_split,
    gradientboost_param_grid,
    n_trials=1
)
print("\nBest Gradient Boosting Accuracy:", best_score_gb)
print("Best Gradient Boosting Params:", best_params_gb)
'''

[03:11:05] INFO: Starting Optuna optimization for 5 trials...
[I 2025-11-15 03:11:05,620] A new study created in memory with name: no-name-066137f2-75fd-459a-bcec-55e24060f257
[03:11:05] INFO: Trial 1/5 - Accuracy: 0.8160 - Params: {'classifier__max_depth': 8, 'classifier__learning_rate': 0.2, 'classifier__subsample': 0.7, 'classifier__colsample_bytree': 0.8, 'classifier__min_child_weight': 7, 'classifier__gamma': 3, 'classifier__reg_alpha': 1, 'classifier__reg_lambda': 0.01, 'classifier__n_estimators': 100}
[I 2025-11-15 03:11:05,814] Trial 0 finished with value: 0.816 and parameters: {'classifier__max_depth': 8, 'classifier__learning_rate': 0.2, 'classifier__subsample': 0.7, 'classifier__colsample_bytree': 0.8, 'classifier__min_child_weight': 7, 'classifier__gamma': 3, 'classifier__reg_alpha': 1, 'classifier__reg_lambda': 0.01, 'classifier__n_estimators': 100}. Best is trial 0 with value: 0.816.
[03:11:06] INFO: Trial 2/5 - Accuracy: 0.8130 - Params: {'classifier__max_depth': 7, 'cla


Best XGBoost Accuracy: 0.823
Best XGBoost Params: {'classifier__max_depth': 6, 'classifier__learning_rate': 0.05, 'classifier__subsample': 0.6, 'classifier__colsample_bytree': 1.0, 'classifier__min_child_weight': 10, 'classifier__gamma': 3, 'classifier__reg_alpha': 0.001, 'classifier__reg_lambda': 0.001, 'classifier__n_estimators': 400}
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] lambda_l2 is set=10.0, reg_lambda=0.0 will be ignored. Current value: lambda_l2=10.0
[LightGBM] [Warning] lambda_l1 is set=0.01, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.01
[LightGBM] [Warning] bagging_fraction is set=0.7, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] lambda_l2 is set=10.0, reg_lambda=0.0 will be ignored. Current 

[03:11:14] INFO: Trial 1/5 - Accuracy: 0.8120 - Params: {'classifier__boosting_type': 'dart', 'classifier__n_estimators': 800, 'classifier__learning_rate': 0.1, 'classifier__num_leaves': 63, 'classifier__max_depth': -1, 'classifier__feature_fraction': 0.8, 'classifier__bagging_fraction': 0.7, 'classifier__min_child_samples': 50, 'classifier__lambda_l1': 0.01, 'classifier__lambda_l2': 10.0}
[I 2025-11-15 03:11:14,839] Trial 0 finished with value: 0.812 and parameters: {'classifier__boosting_type': 'dart', 'classifier__n_estimators': 800, 'classifier__learning_rate': 0.1, 'classifier__num_leaves': 63, 'classifier__max_depth': -1, 'classifier__feature_fraction': 0.8, 'classifier__bagging_fraction': 0.7, 'classifier__min_child_samples': 50, 'classifier__lambda_l1': 0.01, 'classifier__lambda_l2': 10.0}. Best is trial 0 with value: 0.812.


[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] lambda_l2 is set=10.0, reg_lambda=0.0 will be ignored. Current value: lambda_l2=10.0
[LightGBM] [Warning] lambda_l1 is set=0.01, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.01
[LightGBM] [Warning] bagging_fraction is set=0.7, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] lambda_l2 is set=10.0, reg_lambda=0.0 will be ignored. Current value: lambda_l2=10.0
[LightGBM] [Warning] lambda_l1 is set=1.0, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1.0
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current va

[03:11:18] INFO: Trial 2/5 - Accuracy: 0.8080 - Params: {'classifier__boosting_type': 'dart', 'classifier__n_estimators': 700, 'classifier__learning_rate': 0.2, 'classifier__num_leaves': 31, 'classifier__max_depth': -1, 'classifier__feature_fraction': 0.8, 'classifier__bagging_fraction': 0.8, 'classifier__min_child_samples': 30, 'classifier__lambda_l1': 1.0, 'classifier__lambda_l2': 10.0}
[I 2025-11-15 03:11:18,676] Trial 1 finished with value: 0.808 and parameters: {'classifier__boosting_type': 'dart', 'classifier__n_estimators': 700, 'classifier__learning_rate': 0.2, 'classifier__num_leaves': 31, 'classifier__max_depth': -1, 'classifier__feature_fraction': 0.8, 'classifier__bagging_fraction': 0.8, 'classifier__min_child_samples': 30, 'classifier__lambda_l1': 1.0, 'classifier__lambda_l2': 10.0}. Best is trial 0 with value: 0.812.


...

[03:11:23] INFO: Trial 4/5 - Accuracy: 0.8150 - Params: {'classifier__boosting_type': 'dart', 'classifier__n_estimators': 600, 'classifier__learning_rate': 0.2, 'classifier__num_leaves': 255, 'classifier__max_depth': 3, 'classifier__feature_fraction': 1.0, 'classifier__bagging_fraction': 0.7, 'classifier__min_child_samples': 50, 'classifier__lambda_l1': 10.0, 'classifier__lambda_l2': 0.01}
[I 2025-11-15 03:11:23,635] Trial 3 finished with value: 0.815 and parameters: {'classifier__boosting_type': 'dart', 'classifier__n_estimators': 600, 'classifier__learning_rate': 0.2, 'classifier__num_leaves': 255, 'classifier__max_depth': 3, 'classifier__feature_fraction': 1.0, 'classifier__bagging_fraction': 0.7, 'classifier__min_child_samples': 50, 'classifier__lambda_l1': 10.0, 'classifier__lambda_l2': 0.01}. Best is trial 3 with value: 0.815.


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[03:11:26] INFO: Trial 5/5 - Accuracy: 0.8150 - Params: {'classifier__boosting_type': 'dart', 'classifier__n_estimators': 400, 'classifier__learning_rate': 0.15, 'classifier__num_leaves': 63, 'classifier__max_depth': 12, 'classifier__feature_fraction': 0.8, 'classifier__bagging_fraction': 1.0, 'classifier__min_child_samples': 20, 'classifier__lambda_l1': 5.0, 'classifier__lambda_l2': 0.001}
[I 2025-11-15 03:11:26,089] Trial 4 finished with value: 0.815 and parameters: {'classifier__boosting_type': 'dart', 'classifier__n_estimators': 400, 'classifier__learning_rate': 0.15, 'classifier__num_leaves': 63, 'classifier__max_depth': 12, 'classifier__feature_fraction': 0.8, 'classifier__bagging_fraction': 1.0, 'classifier__min_child_samples': 20, 'classifier__lambda_l1': 5.0, 'classifier__lambda_l2': 0.001}. Best is trial 3 with value: 0.815.
[03:11:26] INFO: Optimization finished. Best Accuracy: 0.8150
[03:11:26] INFO: Best Parameters: {'classifier__boosting_type': 'dart', 'classifier__n_esti

[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] lambda_l2 is set=0.001, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.001
[LightGBM] [Warning] lambda_l1 is set=5.0, reg_alpha=0.0 will be ignored. Current value: lambda_l1=5.0
[LightGBM] [Warning] bagging_fraction is set=1.0, subsample=1.0 will be ignored. Current value: bagging_fraction=1.0
[LightGBM] [Warning] feature_fraction is set=1.0, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=1.0
[LightGBM] [Warning] lambda_l2 is set=0.01, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.01
[LightGBM] [Warning] lambda_l1 is set=10.0, reg_alpha=0.0 will be ignored. Current value: lambda_l1=10.0
[LightGBM] [Warning] bagging_fraction is set=0.7, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7
[LightGBM] [Warning] feature_fraction is set=1.0, colsample_bytree=1.0 will be ignored. Current 

'\n# --- GRADIENT BOOSTING ---\nbest_gb_model, best_params_gb, best_score_gb = optimize_optuna(\n    lambda: pipeline_gradient_boost,\n    X_train_split,\n    y_train_split,\n    X_val_split,\n    y_val_split,\n    gradientboost_param_grid,\n    n_trials=1\n)\nprint("\nBest Gradient Boosting Accuracy:", best_score_gb)\nprint("Best Gradient Boosting Params:", best_params_gb)\n'

In [10]:
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

# --- Base models (already fitted) ---
xgb_clf = best_model.named_steps["classifier"]
lgbm_clf = best_lgbm_model.named_steps["classifier"]

# --- Final estimator with scaling to help convergence ---
final_estimator = make_pipeline(
    StandardScaler(),
    LogisticRegression(max_iter=5000, solver="saga", random_state=42)
)

# --- Stacking classifier ---
stacking_model = StackingClassifier(
    estimators=[
        ("xgb", xgb_clf),
        ("lgbm", lgbm_clf)
    ],
    final_estimator=final_estimator,
    passthrough=False,  # only use base model predictions, easier to converge
    n_jobs=-1,
    cv=5  # cross-validated predictions for the final estimator
)

# --- Fit stacking model with preprocessed features ---
X_train_preprocessed = best_model.named_steps["preprocessor"].transform(X_train_split)
X_val_preprocessed = best_model.named_steps["preprocessor"].transform(X_val_split)

stacking_model.fit(X_train_preprocessed, y_train_split)

# --- Predict and evaluate ---
stack_preds = stacking_model.predict(X_val_preprocessed)
from sklearn.metrics import accuracy_score
stack_acc = accuracy_score(y_val_split, stack_preds)
print(f"Stacking Ensemble Validation Accuracy: {stack_acc:.4f}")


[LightGBM] [Warning] feature_fraction is set=1.0, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=1.0
[LightGBM] [Warning] lambda_l2 is set=0.01, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.01
[LightGBM] [Warning] lambda_l1 is set=10.0, reg_alpha=0.0 will be ignored. Current value: lambda_l1=10.0
[LightGBM] [Warning] bagging_fraction is set=0.7, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7
[LightGBM] [Warning] feature_fraction is set=1.0, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=1.0
[LightGBM] [Warning] lambda_l2 is set=0.01, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.01
[LightGBM] [Warning] lambda_l1 is set=10.0, reg_alpha=0.0 will be ignored. Current value: lambda_l1=10.0
[LightGBM] [Warning] bagging_fraction is set=0.7, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7
[LightGBM] [Info] Number of positive: 3998, number of negative: 3998
[LightGBM] [Info] Auto-choo

In [11]:
final_model = stacking_model

from Submission.submit import save_submission
import pandas as pd

X_test_for_submission = X_test_features[features] 

# Agora o 'final_model' receberá as 58 features que ele espera.
save_submission(X_test_for_submission, final_model)

[LightGBM] [Warning] feature_fraction is set=1.0, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=1.0
[LightGBM] [Warning] lambda_l2 is set=0.01, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.01
[LightGBM] [Warning] lambda_l1 is set=10.0, reg_alpha=0.0 will be ignored. Current value: lambda_l1=10.0
[LightGBM] [Warning] bagging_fraction is set=0.7, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7
[INFO] Submission created: submissions/StackingClassifier_20251115_031300.csv


'submissions/StackingClassifier_20251115_031300.csv'